In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import scipy.linalg

from graph_utils import graph_gen
from problem import ElecVehCharg
from plot_utils import MyFigure

from algorithms import UDC
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')


In [12]:
# generate problem

# network

num_node = 50
num_edge_list = [55, 60, 70, 90, 120, 200, 500, 1000]

# graph_gen(num_node, num_edge, B=1, ring=True)
for num_edge in num_edge_list:
    graph_gen(num_node, num_edge, B=1)

net_dir = f'instance/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)


Graph with 50 nodes and 55 edges


c:\Users\xtq\OneDrive - shanghaitech.edu.cn\Research\Dual AMM\numerical\exp2_elec_veh_charg\graph_utils.py:86: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


Graph with 50 nodes and 60 edges
Graph with 50 nodes and 70 edges
Graph with 50 nodes and 90 edges
Graph with 50 nodes and 120 edges
Graph with 50 nodes and 200 edges
Graph with 50 nodes and 500 edges
Graph with 50 nodes and 1000 edges
(50, 50)


In [ ]:
# problem data
num_node = 50
prob = ElecVehCharg(num_node)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')

In [17]:
# load instance 
num_node = 50
instance_name = 'instance1'
instance_dir = 'instance/' + instance_name

# network
num_edge = 120
network = np.load(f'instance/graph/N50E{num_edge}/subgraph_W.npy')
print(network.shape)

# problem data
prob = ElecVehCharg(num_node)
prob.save_dir = instance_dir + '/problem'
prob.load()

print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')


(50, 50)
loading a ElecVehCharg problem, N=50
problem loaded:
c (50,)
d (50,)
b (1,)
x_star (50,)
opt_val 1.8027609655212484
x* [ 9.81e-01 -2.02e-12  1.37e-11  4.92e-11  5.80e-12  1.00e+00  3.74e-12
  1.68e-12  6.11e-12  7.12e-12  1.47e-11 -7.63e-13  1.00e+00 -4.76e-12
  1.90e-11  3.56e-12  5.18e-02  1.38e-12  4.42e-13 -7.67e-13  1.08e-12
  5.26e-12  6.20e-01 -4.55e-12  7.67e-12  8.38e-02 -1.32e-12  1.00e+00
 -3.66e-12  1.00e+00  9.58e-01  6.76e-12  3.63e-01  1.35e-11  1.31e-12
  1.00e+00  8.03e-11  4.12e-12  1.80e-11  2.50e-11  5.83e-11 -2.84e-12
  7.08e-11  2.07e-11 -1.14e-12  1.00e+00  1.40e-11 -1.47e-12  4.70e-11
  5.91e-01]
||x*|| 2.958778115102063



In [51]:
# UDC
# effect of edges

# best parameters (for 120 edge graph):
# PEXTRA: r1
# PGC: r0.1
# D.A.: 0.5
# ALT: 2

# MAX_ITER = 500
MAX_ITER = 1000
# MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

num_edge_list = [50, 60, 90, 120, 200, 500, 1000]
# num_edge = 120

# rho_choice = [0.1, 1, 10, 100]
# rho_choice = [0.5, 1, 2, 4]
# rho_choice = [0.25, 0.5]
rho_choice = [1]
# rho_choice = [0.1]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']
al_list = [0.1*i for i in range(10, 0, -1)]

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')
    
np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})

for num_edge in num_edge_list:
# for rho in rho_choice:
    
    ci = 1
    ali = num_edge_list.index(num_edge)
    # ci = rho_choice.index(rho)
    # ali = 1
    
    c = color[ci]
    al = al_list[ali]
    
    network = np.load(f'instance/graph/N50E{num_edge}/subgraph_W.npy')
    
    rho = 1
    if True:
    # if alpha != 0:
    # if rho != 1:
    # if rho == 4:
        alg = UDC(prob, network, rho=rho, alpha=0,
                    param_setting='PEXTRA', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='PGC', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='DPGA', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='DistADMM', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=0,
        #         param_setting='ALT', verbose=False)
        for i in range(MAX_ITER):
            alg.step()
        
    prefix = f'{alg.name}_a{0}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(f'PEXTRA_r{rho}_e{num_edge}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'PEXTRA_r{rho}_e{num_edge}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'PEXTRA_r{rho}_e{num_edge}', filename_xd, style=c, alpha=al)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    
    
    
    # c = color[2]
    # rho =0.1
    # if True:
    # # if alpha != 0:
    # # if rho != 1:
    # # if rho == 4:
    #     # alg = UDC(prob, network, rho=rho, alpha=0,
    #     #           param_setting='PEXTRA', verbose=False)
    #     alg = UDC(prob, network, rho=rho, alpha=0,
    #                 param_setting='PGC', verbose=False)
    #     # alg = UDC(prob, network, rho=rho, alpha=alpha,
    #     #           param_setting='DPGA', verbose=False)
    #     # alg = UDC(prob, network, rho=rho, alpha=alpha,
    #     #           param_setting='DistADMM', verbose=False)
    #     # alg = UDC(prob, network, rho=rho, alpha=alpha,
    #     #         param_setting='ALT', verbose=False)
    #     for i in range(MAX_ITER):
    #         alg.step()
        
    # prefix = f'{alg.name}_a{0}_r{rho}'
    # filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    # filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    # filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    # obj_err_figure.add_line_file(f'PGC_r{rho}_e{num_edge}', filename_oe, style=c, alpha=al)
    # cons_vio_figure.add_line_file(f'PGC_r{rho}_e{num_edge}', filename_cv, style=c, alpha=al)
    # x_dis_figure.add_line_file(f'PGC_r{rho}_e{num_edge}', filename_xd, style=c, alpha=al)
        
    # obj_err_figure.paint(MAX_ITER=MAX_ITER)
    # cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    # x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

    c = color[3]
    rho =0.5
    if True:
    # if alpha != 0:
    # if rho != 1:
    # if rho == 4:
        # alg = UDC(prob, network, rho=rho, alpha=0,
        #           param_setting='PEXTRA', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='PGC', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='DPGA', verbose=False)
        alg = UDC(prob, network, rho=0.5, alpha=0,
                    param_setting='DistADMM', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #         param_setting='ALT', verbose=False)
        for i in range(MAX_ITER):
            alg.step()
        
    prefix = f'{alg.name}_a{0}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(f'D.A._r{rho}_e{num_edge}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'D.A._r{rho}_e{num_edge}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'D.A._r{rho}_e{num_edge}', filename_xd, style=c, alpha=al)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    
    
    c = color[4]
    rho =2
    if True:
    # if alpha != 0:
    # if rho != 1:
    # if rho == 4:
        # alg = UDC(prob, network, rho=rho, alpha=0,
        #           param_setting='PEXTRA', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='PGC', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='DPGA', verbose=False)
        # alg = UDC(prob, network, rho=0.5, alpha=0,
        #             param_setting='DistADMM', verbose=False)
        alg = UDC(prob, network, rho=rho, alpha=0,
                param_setting='ALT', verbose=False)
        for i in range(MAX_ITER):
            alg.step()
        
    prefix = f'{alg.name}_a{0}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(f'ALT_r{rho}_e{num_edge}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'ALT_r{rho}_e{num_edge}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'ALT_r{rho}_e{num_edge}', filename_xd, style=c, alpha=al)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

UDC setting: PEXTRA
d: (50,)
reset
UDC_PEXTRA alpha 0 rho 1, iter 0, obj err: 1.80e+00, cons vio: 5.00e+00
time 0.00, saved

UDC_PEXTRA alpha 0 rho 1, iter 100, obj err: 2.78e-02, cons vio: 0.00e+00
time 0.59, saved

UDC_PEXTRA alpha 0 rho 1, iter 200, obj err: 1.40e-03, cons vio: 2.73e-03
time 1.17, saved

UDC_PEXTRA alpha 0 rho 1, iter 300, obj err: 8.15e-04, cons vio: 0.00e+00
time 1.78, saved

UDC_PEXTRA alpha 0 rho 1, iter 400, obj err: 9.48e-05, cons vio: 1.64e-04
time 2.34, saved

UDC_PEXTRA alpha 0 rho 1, iter 500, obj err: 6.91e-05, cons vio: 1.18e-04
time 2.91, saved

UDC_PEXTRA alpha 0 rho 1, iter 600, obj err: 2.00e-05, cons vio: 0.00e+00
time 3.54, saved

UDC_PEXTRA alpha 0 rho 1, iter 700, obj err: 4.09e-06, cons vio: 6.98e-06
time 4.11, saved

UDC_PEXTRA alpha 0 rho 1, iter 800, obj err: 5.09e-07, cons vio: 8.68e-07
time 4.69, saved

UDC_PEXTRA alpha 0 rho 1, iter 900, obj err: 8.62e-08, cons vio: 0.00e+00
time 5.30, saved

UDC_PEXTRA alpha 0 rho 1, iter 1000, obj err: 1

In [136]:
# UDC
# effect of alpha

# best parameters (for 120 edge graph):
# PEXTRA: r1
# PGC: r0.1
# DPGA: r0.1
# D.A.: 0.5
# ALT: 2

# MAX_ITER = 500
MAX_ITER = 1000
# MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# num_edge_list = [50, 60, 90, 120, 200, 500, 1000]
num_edge = 120

# rho_choice = [0.1, 1, 10, 100]
# rho_choice = [0.5, 1, 2, 4]
# rho_choice = [0.25, 0.5]
# rho_choice = [1]
rho_choice = [0.05, 0.1, 0.2]
# alpha_choice = [0, 0.1, 1]
alpha_choice = [0.2, 2]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']
al_list = [0.2, 0.5, 1]

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')
    
np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})

# for num_edge in num_edge_list:
# for rho in rho_choice:
for alpha in alpha_choice:
    network = np.load(f'instance/graph/N50E{num_edge}/subgraph_W.npy')
    
    ali = alpha_choice.index(alpha)
    al = al_list[ali]
    
    c = color[1]
    rho = 1
    alg = UDC(prob, network, rho=rho, alpha=alpha,
                    param_setting='PEXTRA', verbose=False)
    for i in range(MAX_ITER):
        alg.step()
        
    prefix = f'{alg.name}_a{alpha}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(f'PEXTRA_r{rho}_a{alpha}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'PEXTRA_r{rho}_a{alpha}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'PEXTRA_r{rho}_a{alpha}', filename_xd, style=c, alpha=al)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    
    
    
    c = color[2]
    rho =0.1
    alg = UDC(prob, network, rho=rho, alpha=alpha,
                    param_setting='PGC', verbose=False)
    for i in range(MAX_ITER):
        alg.step()
        
    prefix = f'{alg.name}_a{alpha}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    obj_err_figure.add_line_file(f'PGC_r{rho}_a{alpha}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'PGC_r{rho}_a{alpha}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'PGC_r{rho}_a{alpha}', filename_xd, style=c, alpha=al)
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


    c = color[3]
    rho =0.1
    alg = UDC(prob, network, rho=rho, alpha=alpha,
                  param_setting='DPGA', verbose=False)
    for i in range(MAX_ITER):
        alg.step()
        
    prefix = f'{alg.name}_a{alpha}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    obj_err_figure.add_line_file(f'DPGA_r{rho}_a{alpha}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'DPGA_r{rho}_a{alpha}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'DPGA_r{rho}_a{alpha}', filename_xd, style=c, alpha=al)
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


    c = color[4]
    rho =0.5
    alg = UDC(prob, network, rho=0.5, alpha=alpha,
                    param_setting='DistADMM', verbose=False)
    for i in range(MAX_ITER):
        alg.step()
        
    prefix = f'{alg.name}_a{alpha}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    obj_err_figure.add_line_file(f'D.A._r{rho}_a{alpha}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'D.A._r{rho}_a{alpha}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'D.A._r{rho}_a{alpha}', filename_xd, style=c, alpha=al)
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    
    
    
    c = color[5]
    rho = 2
    alg = UDC(prob, network, rho=rho, alpha=alpha,
                param_setting='ALT', verbose=False)
    for i in range(MAX_ITER):
        alg.step()
        
    prefix = f'{alg.name}_a{alpha}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    obj_err_figure.add_line_file(f'ALT_r{rho}_a{alpha}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'ALT_r{rho}_a{alpha}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'ALT_r{rho}_a{alpha}', filename_xd, style=c, alpha=al)
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

UDC setting: PEXTRA
d: (50,)
reset
UDC_PEXTRA alpha 0.2 rho 1, iter 0, obj err: 1.80e+00, cons vio: 5.00e+00
time 0.02, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 100, obj err: 6.62e-05, cons vio: 0.00e+00
time 0.72, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 200, obj err: 1.89e-07, cons vio: 3.45e-07
time 1.38, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 300, obj err: 1.39e-08, cons vio: 0.00e+00
time 2.03, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 400, obj err: 1.33e-08, cons vio: 1.68e-12
time 2.68, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 500, obj err: 1.33e-08, cons vio: 0.00e+00
time 3.25, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 600, obj err: 1.33e-08, cons vio: 5.59e-12
time 3.89, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 700, obj err: 1.33e-08, cons vio: 0.00e+00
time 4.48, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 800, obj err: 1.33e-08, cons vio: 0.00e+00
time 5.09, saved

UDC_PEXTRA alpha 0.2 rho 1, iter 900, obj err: 1.33e-08, cons vio: 0.00e+00
time 5.72, saved

UDC_PEXTRA alpha 0.2 rho 1,

In [139]:
# load log data

# best parameters (for 120 edge graph):
# PEXTRA: r1
# PGC: r0.1
# DPGA: r0.1
# D.A.: r0.5
# ALT: r2

MAX_ITER = 1000
network = np.load(f'instance/graph/N50E{120}/subgraph_W.npy')
log_dir = f'instance/instance1/log'

alpha_choice = [0, 0.1, 0.2, 1, 2]

oe_logs = dict()
cv_logs = dict()
xd_logs = dict()
oe_cv_logs = dict()

for alpha in alpha_choice:
    # xd
    alg_param_name = f'UDC_PEXTRA_a{alpha}_r{1}'
    filename_xd = f'{log_dir}/{alg_param_name}_xd.txt'
    xd_logs[alg_param_name] = np.loadtxt(filename_xd)

    alg_param_name = f'UDC_PGC_a{alpha}_r{0.1}'
    filename_xd = f'{log_dir}/{alg_param_name}_xd.txt'
    xd_logs[alg_param_name] = np.loadtxt(filename_xd)
    
    alg_param_name = f'UDC_DPGA_a{alpha}_r{0.1}'
    filename_xd = f'{log_dir}/{alg_param_name}_xd.txt'
    xd_logs[alg_param_name] = np.loadtxt(filename_xd)
    
    alg_param_name = f'ALT_a{alpha}_r{2}'
    filename_xd = f'{log_dir}/{alg_param_name}_xd.txt'
    xd_logs[alg_param_name] = np.loadtxt(filename_xd)

    # oe
    alg_param_name = f'UDC_PEXTRA_a{alpha}_r{1}'
    filename_oe = f'{log_dir}/{alg_param_name}_oe.txt'
    oe_logs[alg_param_name] = np.loadtxt(filename_oe)

    alg_param_name = f'UDC_PGC_a{alpha}_r{0.1}'
    filename_oe = f'{log_dir}/{alg_param_name}_oe.txt'
    oe_logs[alg_param_name] = np.loadtxt(filename_oe)
    
    alg_param_name = f'UDC_DPGA_a{alpha}_r{0.1}'
    filename_oe = f'{log_dir}/{alg_param_name}_oe.txt'
    oe_logs[alg_param_name] = np.loadtxt(filename_oe)
    
    alg_param_name = f'ALT_a{alpha}_r{2}'
    filename_oe = f'{log_dir}/{alg_param_name}_oe.txt'
    oe_logs[alg_param_name] = np.loadtxt(filename_oe)
    
    # cv
    alg_param_name = f'UDC_PEXTRA_a{alpha}_r{1}'
    filename_cv = f'{log_dir}/{alg_param_name}_cv.txt'
    cv_logs[alg_param_name] = np.loadtxt(filename_cv)

    alg_param_name = f'UDC_PGC_a{alpha}_r{0.1}'
    filename_cv = f'{log_dir}/{alg_param_name}_cv.txt'
    cv_logs[alg_param_name] = np.loadtxt(filename_cv)
    
    alg_param_name = f'UDC_DPGA_a{alpha}_r{0.1}'
    filename_cv = f'{log_dir}/{alg_param_name}_cv.txt'
    cv_logs[alg_param_name] = np.loadtxt(filename_cv)
    
    alg_param_name = f'ALT_a{alpha}_r{2}'
    filename_cv = f'{log_dir}/{alg_param_name}_cv.txt'
    cv_logs[alg_param_name] = np.loadtxt(filename_cv)
    
    # oe + cv
    alg_param_name = f'UDC_PEXTRA_a{alpha}_r{1}'
    filename_oe = f'{log_dir}/{alg_param_name}_oe.txt'
    filename_cv = f'{log_dir}/{alg_param_name}_cv.txt'
    oe_cv_logs[alg_param_name] = np.loadtxt(filename_oe)+np.loadtxt(filename_cv)

    alg_param_name = f'UDC_PGC_a{alpha}_r{0.1}'
    filename_oe = f'{log_dir}/{alg_param_name}_oe.txt'
    filename_cv = f'{log_dir}/{alg_param_name}_cv.txt'
    oe_cv_logs[alg_param_name] = np.loadtxt(filename_oe)+np.loadtxt(filename_cv)
    
    alg_param_name = f'UDC_DPGA_a{alpha}_r{0.1}'
    filename_oe = f'{log_dir}/{alg_param_name}_oe.txt'
    filename_cv = f'{log_dir}/{alg_param_name}_cv.txt'
    oe_cv_logs[alg_param_name] = np.loadtxt(filename_oe)+np.loadtxt(filename_cv)
    
    alg_param_name = f'ALT_a{alpha}_r{2}'
    filename_oe = f'{log_dir}/{alg_param_name}_oe.txt'
    filename_cv = f'{log_dir}/{alg_param_name}_cv.txt'
    oe_cv_logs[alg_param_name] = np.loadtxt(filename_oe)+np.loadtxt(filename_cv)

In [169]:
# fig 1: effect of alpha
# subfig: x_distance

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
from matplotlib.ticker import FormatStrFormatter, LogFormatter
from matplotlib.legend_handler import HandlerTuple
# rc('mathtext', fontset='stixsans')
# rc('text', usetex=True)
# rc('text', usetex=True)
# rc('mathtext', fontset ='dejavusans')
# rc('font', **{'family': 'sans-serif', 'sans-serif': ['DejaVu Sans'], 'size': 40})

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": "Computer Modern Roman",
    "font.sans-serif": "Helvetica",
    "font.size": 42,
    'text.latex.preamble': r'\usepackage{sfmath}' # sans-serif for math
})


MAX_ITER = 500
fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'$\Vert x^k-x^\star\Vert$', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


# alpha_choice = [0, 0.1, 0.2, 1, 2]
alpha_choice = [0, 0.2, 2]
darkness = {
        0  : 1,
        0.1: 0.5,
        0.2: 0.5,
        1  : 0.8,
        2  : 1
    }
ls = {
        0  : ':',
        0.1: '-',
        0.2: '-',
        1  : '-',
        2  : '-'
    }
c = {
    
}

p1 = {}
p2 = {}
p3 = {}
p4 = {}
for alpha in alpha_choice:
    alg_param_name = f'UDC_PEXTRA_a{alpha}_r{1}'
    p1[alpha], = ax.plot(xd_logs[alg_param_name], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    c='tab:purple', 
    alpha=darkness[alpha], 
    ls=ls[alpha])
    
    # alg_param_name = f'UDC_PGC_a{alpha}_r{0.1}'
    # p2[alpha], = ax.plot(xd_logs[alg_param_name], label=alg_param_name, 
    # lw=3,
    # # lw = lw[alpha], 
    # c='xkcd:green', 
    # alpha=darkness[alpha], 
    # ls=ls[alpha]) 
    
    alg_param_name = f'UDC_DPGA_a{alpha}_r{0.1}'
    p3[alpha], = ax.plot(xd_logs[alg_param_name], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    c='tab:blue', 
    alpha=darkness[alpha], 
    ls=ls[alpha]) 
    
    alg_param_name = f'ALT_a{alpha}_r{2}'
    p4[alpha], = ax.plot(xd_logs[alg_param_name], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    c='tab:orange', 
    alpha=darkness[alpha], 
    ls=ls[alpha])

# ax.legend([(p1[0], p1[0.1], p1[0.2], p1[1], p1[2]), 
#         #    (p2[0], p2[0.1], p2[0.2], p2[1], p2[2]),
#            (p3[0], p3[0.1], p3[0.2], p3[1], p3[2]),
#            (p4[0], p4[0.1], p4[0.2], p4[1], p4[2])], 
#           ['ProxUDC-PEXTRA', 
#         #    'ProxUDC-PGC', 
#            'ProxUDC-DPGA', 
#            'ProxUDC-ALT'], 
#             handler_map={tuple: HandlerTuple(ndivide=None)},
#             fontsize=40)
ax.legend([ p1[0], p3[0], p4[0],
           (p1[0.2], p1[2]), 
           (p3[0.2], p3[2]),
           (p4[0.2], p4[2])], 
          [ 'UDC-PEXTRA',
            'UDC-DPGA', 
            'UDC-ALT',
            'ProxUDC-PEXTRA', 
            'ProxUDC-DPGA', 
            'ProxUDC-ALT'], 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            fontsize=40)
# ax.legend(fontsize=40)
fig.savefig(f'compare_x_distance.pdf', bbox_inches='tight', transparent=False)




In [170]:
# fig 1: effect of alpha
# subfig: objective error + constraint violation


MAX_ITER = 500
fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'Objective Error + Constraint Violation', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')

# alpha_choice = [0, 0.1, 0.2, 1, 2]
alpha_choice = [0, 0.2, 2]
darkness = {
        0  : 1,
        0.1: 0.5,
        0.2: 0.5,
        1  : 0.8,
        2  : 1
    }
ls = {
        0  : ':',
        0.1: '-',
        0.2: '-',
        1  : '-',
        2  : '-'
    }
c = {
    
}

p1 = {}
p2 = {}
p3 = {}
p4 = {}
for alpha in alpha_choice:
    alg_param_name = f'UDC_PEXTRA_a{alpha}_r{1}'
    p1[alpha], = ax.plot(oe_cv_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='tab:purple'
    )
    
    # alg_param_name = f'UDC_PGC_a{alpha}_r{0.1}'
    # p2[alpha], = ax.plot(oe_cv_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    # lw=3,
    # # lw = lw[alpha], 
    # alpha=darkness[alpha], 
    # ls=ls[alpha],
    # c='xkcd:blue'
    # ) 
    
    alg_param_name = f'UDC_DPGA_a{alpha}_r{0.1}'
    p3[alpha], = ax.plot(oe_cv_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='tab:blue'
    ) 
    
    alg_param_name = f'ALT_a{alpha}_r{2}'
    p4[alpha], = ax.plot(oe_cv_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='tab:orange'
    )

ax.legend([ p1[0], p3[0], p4[0],
           (p1[0.2], p1[2]), 
           (p3[0.2], p3[2]),
           (p4[0.2], p4[2])], 
          [ 'UDC-PEXTRA',
            'UDC-DPGA', 
            'UDC-ALT',
            'ProxUDC-PEXTRA', 
            'ProxUDC-DPGA', 
            'ProxUDC-ALT'], 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            fontsize=40)
# ax.legend(fontsize=40)
fig.savefig(f'compare_oe_cv.pdf', bbox_inches='tight', transparent=False)

In [119]:
# fig 1: effect of alpha
# subfig: objective error

MAX_ITER = 500
fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'Objective Error', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


alpha_choice = [0, 0.1, 1]
darkness = {
        0  : 1,
        0.1: 0.6,
        1  : 0.6
    }
ls = {
        0  : '-',
        0.1: '-.',
        1  : ':'
    }
lw = {
        0  : 3,
        0.1: 5,
        1  : 7
    }

for alpha in alpha_choice:
    alg_param_name = f'UDC_PEXTRA_a{alpha}_r{1}'
    p1, = ax.plot(oe_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='xkcd:goldenrod'
    )
    
    alg_param_name = f'UDC_PGC_a{alpha}_r{0.1}'
    p2, = ax.plot(oe_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='xkcd:green'
    ) 
    
    alg_param_name = f'UDC_DPGA_a{alpha}_r{0.1}'
    p3, = ax.plot(oe_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='xkcd:blue'
    ) 
    
    alg_param_name = f'ALT_a{alpha}_r{2}'
    p4, = ax.plot(oe_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='xkcd:orangered'
    )

# ax.legend([(p1, p2, p3), p4, p5, p6, p7, p8], 
#           ['UDC-SingleRound', 'UDC-Dist.ADMM', 'ALT', 'DPMM', 
#            'IPLUX', 'dual subgradient'], 
#             handler_map={tuple: HandlerTuple(ndivide=None)},
#             fontsize=40)
# ax.legend(fontsize=40)
fig.savefig(f'compare_obj_err.pdf', bbox_inches='tight', transparent=False)

In [118]:
# fig 1: effect of alpha
# subfig: constraint violation

MAX_ITER = 500
fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'Constriant Violation', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


alpha_choice = [0, 0.1, 1]
darkness = {
        0  : 1,
        0.1: 0.6,
        1  : 0.6
    }
ls = {
        0  : '-',
        0.1: '-.',
        1  : ':'
    }
lw = {
        0  : 3,
        0.1: 5,
        1  : 7
    }

for alpha in alpha_choice:
    alg_param_name = f'UDC_PEXTRA_a{alpha}_r{1}'
    p1, = ax.plot(cv_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='xkcd:goldenrod'
    )
    
    alg_param_name = f'UDC_PGC_a{alpha}_r{0.1}'
    p2, = ax.plot(cv_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='xkcd:green'
    ) 
    
    alg_param_name = f'UDC_DPGA_a{alpha}_r{0.1}'
    p3, = ax.plot(cv_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='xkcd:blue'
    ) 
    
    alg_param_name = f'ALT_a{alpha}_r{2}'
    p4, = ax.plot(cv_logs[alg_param_name][:MAX_ITER], label=alg_param_name, 
    lw=3,
    # lw = lw[alpha], 
    alpha=darkness[alpha], 
    ls=ls[alpha],
    c='xkcd:orangered'
    )

# ax.legend([(p1, p2, p3), p4, p5, p6, p7, p8], 
#           ['UDC-SingleRound', 'UDC-Dist.ADMM', 'ALT', 'DPMM', 
#            'IPLUX', 'dual subgradient'], 
#             handler_map={tuple: HandlerTuple(ndivide=None)},
#             fontsize=40)
# ax.legend(fontsize=40)
fig.savefig(f'compare_cons_vio.pdf', bbox_inches='tight', transparent=False)